<a href="https://colab.research.google.com/github/junghoum/Hello-world/blob/main/n221a_decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 1*

# 📝 Assignment
---

# Decision Trees(결정트리)

## 캐글 인클래스(InClass) Competitions

### 1) 대회 참여(Join) 및 데이터셋 준비

- [캐글 계정을 만드세요](https://www.kaggle.com/), 공유된 인클래스 대회 url을 통해 이동한 후 대회에 참여하세요.
- H1N1 데이터를 훈련/검증/테스트 셋으로 나누어 준비하세요
- **(객관식) 훈련데이터셋을 사용해 H1N1 타겟으로 기준모델을 설정하세요. 기준모델의 정확도를 제출하세요.**
    1. 0.24
    2. 0.50
    3. 0.76
    4. 1


In [1]:
### 이곳에서 과제를 진행해 주세요 ###
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.jason

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.jason': No such file or directory


In [2]:
!kaggle competitions download -c prediction-of-h1n1-vaccination

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [3]:
# from google.colab import files
# from google.colab import drive
# import io
# import pandas as pd

# drive.mount('/content/drive')

In [4]:
import pandas as pd

target = 'vacc_h1n1_f'
# target = 'vacc_seas_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [5]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)


train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [6]:
train[target].value_counts(normalize=True)

0    0.760935
1    0.239065
Name: vacc_h1n1_f, dtype: float64

### 2) 데이터 EDA 및 특성공학 수행
- profiling을 사용하여 데이터를 살펴보고, missing value, zeros, cardinarity 등을 확인하여 처리하세요.
- 본인만의 특성공학을 수행하세요.


In [7]:
!pip3 install pandas_profiling --upgrade

In [8]:
### 이곳에서 과제를 진행해 주세요 ###
from pandas_profiling import ProfileReport as pr
profile = pr(train, minimal=True).to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
train.T.duplicated()
#중복된 특성 확인

h1n1_concern                   False
h1n1_knowledge                 False
behavioral_antiviral_meds      False
behavioral_avoidance           False
behavioral_face_mask           False
behavioral_wash_hands          False
behavioral_large_gatherings    False
behavioral_outside_home        False
behavioral_touch_face          False
doctor_recc_h1n1               False
doctor_recc_seasonal           False
chronic_med_condition          False
child_under_6_months           False
health_insurance               False
health_worker                  False
opinion_h1n1_vacc_effective    False
opinion_h1n1_risk              False
opinion_h1n1_sick_from_vacc    False
opinion_seas_vacc_effective    False
opinion_seas_risk              False
opinion_seas_sick_from_vacc    False
agegrp                         False
education_comp                 False
raceeth4_i                     False
sex_i                          False
inc_pov                        False
marital                        False
r

In [10]:
train.describe(exclude='number').T.sort_values(by='unique')
#cardinality확인

,count,unique,top,freq
employment_status,25466,3,Employed,13722
census_msa,33723,3,"MSA, Not Principle City",15202
opinion_h1n1_vacc_effective,26637,6,Somewhat Effective,11710
opinion_h1n1_risk,26602,6,Somewhat Low,9980
opinion_h1n1_sick_from_vacc,26568,6,Not Very Worried,9229
opinion_seas_vacc_effective,26519,6,Somewhat Effective,11571
opinion_seas_risk,26466,6,Somewhat Low,9057
opinion_seas_sick_from_vacc,26413,6,Not At All Worried,11813
agegrp,33723,7,65+ Years,6821
employment_industry,13714,23,Health Care and Social Assistance Industries,2451


In [11]:
train.isnull().sum()

h1n1_concern                    6790
h1n1_knowledge                  6790
behavioral_antiviral_meds       6870
behavioral_avoidance            6992
behavioral_face_mask            6810
behavioral_wash_hands           6829
behavioral_large_gatherings     6857
behavioral_outside_home         6872
behavioral_touch_face           6928
doctor_recc_h1n1                1488
doctor_recc_seasonal            1488
chronic_med_condition           1075
child_under_6_months            7617
health_insurance               15505
health_worker                   7603
opinion_h1n1_vacc_effective     7086
opinion_h1n1_risk               7121
opinion_h1n1_sick_from_vacc     7155
opinion_seas_vacc_effective     7204
opinion_seas_risk               7257
opinion_seas_sick_from_vacc     7310
agegrp                             0
education_comp                  8183
raceeth4_i                         0
sex_i                              0
inc_pov                            0
marital                         8199
r

In [12]:
train.dropna()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_insurance,health_worker,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,agegrp,education_comp,raceeth4_i,sex_i,inc_pov,marital,rent_own_r,employment_status,census_region,census_msa,n_adult_r,household_children,n_people_r,employment_industry,employment_occupation,hhs_region,state,vacc_h1n1_f
39793,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Somewhat Low,Not Very Worried,Somewhat Effective,Very Low,Not At All Worried,55 - 64 Years,4.0,3,1,1,1.0,1.0,Employed,2,"MSA, Principle City",2.0,0.0,2.0,"Arts, Entertainment, and Recreation Industries...",Business and Financial Operations Occupations,7,IOWA,0
30019,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Very Effective,Somewhat High,Somewhat Worried,Very Effective,Somewhat Low,Somewhat Worried,45 - 54 Years,3.0,3,1,2,1.0,2.0,Employed,4,"MSA, Not Principle City",2.0,1.0,3.0,Manufacturing Industries,Production Occupations,9,NEVADA,1
14807,3.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,Very Effective,Somewhat High,Very Worried,Somewhat Effective,Somewhat High,Somewhat Worried,55 - 64 Years,4.0,3,1,1,2.0,1.0,Employed,3,"MSA, Not Principle City",1.0,0.0,1.0,Armed Forces,Military Specific Occupations,4,ALABAMA,1
9029,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,Somewhat Effective,Somewhat Low,Not Very Worried,Very Effective,Very Low,Not Very Worried,45 - 54 Years,2.0,2,2,2,1.0,1.0,Employed,3,Non-MSA,2.0,0.0,2.0,Not ascertained,Office and Administrative Support Occupations,3,MARYLAND,0
19820,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Somewhat Effective,Very Low,Not At All Worried,Somewhat Effective,Somewhat Low,Not At All Worried,45 - 54 Years,4.0,3,1,1,1.0,1.0,Employed,1,"MSA, Not Principle City",2.0,0.0,2.0,Utilities Industries,Management Occupations,2,NEW JERSEY,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8577,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Somewhat Effective,Somewhat Low,Not Very Worried,Not Very Effective,Somewhat Low,Not Very Worried,45 - 54 Years,2.0,3,2,1,1.0,1.0,Employed,2,Non-MSA,2.0,0.0,2.0,Accommodation and Food Services Industries,Management Occupations,8,SOUTH DAKOTA,0
11150,1.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,Very Effective,Somewhat High,Not At All Worried,Very Effective,Somewhat High,Not At All Worried,45 - 54 Years,3.0,3,1,3,1.0,1.0,Employed,2,"MSA, Not Principle City",3.0,3.0,6.0,Health Care and Social Assistance Industries,Healthcare Practitioners and Technical Occupat...,5,INDIANA,1
1790,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Somewhat Effective,Somewhat Low,Not At All Worried,Very Effective,Very High,Somewhat Worried,18 - 34 Years,3.0,3,1,2,2.0,2.0,Employed,4,Non-MSA,3.0,0.0,3.0,Retail Trade Industries,Office and Administrative Support Occupations,8,WYOMING,0
14173,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,Very Effective,Somewhat Low,Not At All Worried,Very Effective,Somewhat High,Somewhat Worried,45 - 54 Years,4.0,3,2,2,2.0,1.0,Employed,4,"MSA, Principle City",1.0,0.0,1.0,Education Services Industries,Healthcare Practitioners and Technical Occupat...,6,NEW MEXICO,1


In [13]:
train.isnull().value_counts()

h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  behavioral_large_gatherings  behavioral_outside_home  behavioral_touch_face  doctor_recc_h1n1  doctor_recc_seasonal  chronic_med_condition  child_under_6_months  health_insurance  health_worker  opinion_h1n1_vacc_effective  opinion_h1n1_risk  opinion_h1n1_sick_from_vacc  opinion_seas_vacc_effective  opinion_seas_risk  opinion_seas_sick_from_vacc  agegrp  education_comp  raceeth4_i  sex_i  inc_pov  marital  rent_own_r  employment_status  census_region  census_msa  n_adult_r  household_children  n_people_r  employment_industry  employment_occupation  hhs_region  state  vacc_h1n1_f
False         False           False                      False                 False                 False                  False                        False                    False                  False             False                 False                  False                 Fals

In [14]:
train1 = train.dropna(axis=0, how='any')
train1.isnull().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_insurance               0
health_worker                  0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
agegrp                         0
education_comp                 0
raceeth4_i                     0
sex_i                          0
inc_pov                        0
marital                        0
rent_own_r                     0
employment_status              0
census_region                  0
census_msa

In [15]:
train2 = train.dropna(axis=1, how='all')
train2.isnull().sum()

h1n1_concern                    6790
h1n1_knowledge                  6790
behavioral_antiviral_meds       6870
behavioral_avoidance            6992
behavioral_face_mask            6810
behavioral_wash_hands           6829
behavioral_large_gatherings     6857
behavioral_outside_home         6872
behavioral_touch_face           6928
doctor_recc_h1n1                1488
doctor_recc_seasonal            1488
chronic_med_condition           1075
child_under_6_months            7617
health_insurance               15505
health_worker                   7603
opinion_h1n1_vacc_effective     7086
opinion_h1n1_risk               7121
opinion_h1n1_sick_from_vacc     7155
opinion_seas_vacc_effective     7204
opinion_seas_risk               7257
opinion_seas_sick_from_vacc     7310
agegrp                             0
education_comp                  8183
raceeth4_i                         0
sex_i                              0
inc_pov                            0
marital                         8199
r

In [17]:
import numpy as np

def engineer(df):
    """특성을 엔지니어링 하는 함수입니다."""
    
    # 높은 카디널리티를 가지는 특성을 제거합니다.
    selected_cols = df.select_dtypes(include=['number', 'object'])
    labels = selected_cols.nunique() # 특성별 카디널리티 리스트
    selected_features = labels[labels <= 30].index.tolist() # 카디널리티가 30보다 작은 특성만 선택합니다.
    df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train = engineer(train)
val = engineer(val)
test = engineer(test)

In [39]:
# 6. 데이터에서 타겟과 특성을 분리
features = train.drop(columns=[target]).columns

# 훈련,검증,테스트로 분리
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape

((33723, 31), (33723,), (8431, 31), (8431,), (28104, 31))

In [21]:
!pip install --upgrade category_encoders



     |████████████████████████████████| 82 kB 324 kB/s 


In [61]:
# 7. 정확도 검증
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score

pipe = make_pipeline(
    OneHotEncoder(), 
    SimpleImputer(), 
    StandardScaler(), 
    LogisticRegression(n_jobs=-1)
)
pipe.fit(X_train, y_train)

print('검증세트 정확도', pipe.score(X_val, y_val))

y_pred = pipe.predict(X_test)





검증세트 정확도 0.8185268651405527


In [62]:
pred = pipe.predict(X_val)
print('F1 score: ', f1_score(y_val, pred))

F1 score:  0.5613532110091743



### 3) 모델링 수행
- 결정트리를 학습시킵니다. 하이퍼파라미터를 수정하여 성능을 높여보세요. 과적합을 주의하세요!
- 검증세트 스코어를 확인하세요.(평가지표는 F1-score입니다, 수식은 캐글 Overview-Evaluation 페이지를 확인해 주세요. 평가지표에 대한 내용은 다음에 자세히 배우고 활용할 것입니다 지금은 스코어를 올리기 위한 노력을 전혀 기울이지 말고 결정트리의 개념과 코드이해, 사용에 집중하세요!)
- 특성중요도를 그래프로 나타내 보세요
- **검증세트의 F1-score를 제출하세요. (0.5 이상 통과)**

In [40]:
pred = pipe.predict(X_val)
print('F1 score: ', f1_score(y_val, pred))

F1 score:  0.5613532110091743


### 4) 테스트셋 예측 결과 제출(Submission)

- 테스트셋에 대한 예측 결과를 캐글에 제출 하세요.(스코어가 많이 낮게 나와도 결정트리를 잘 이해하고 결과를 내었다면 충분합니다!)
- **캐글 Leaderboard의 본인 Score를 제출하세요.**
 

In [60]:
pred_2 = pipe.predict(X_test)

df = pd.DataFrame({'ID':range(0,len(pred_2)),
                   'vacc_h1n1_f':pred_2})

df

,ID,vacc_h1n1_f
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
28099,28099,0
28100,28100,0
28101,28101,0
28102,28102,0


In [57]:
df.set_index('ID',inplace=True)
df

,vacc_h1n1_f
ID,
0,0
1,0
2,0
3,1
4,0
...,...
8426,1
8427,0
8428,0


In [59]:
df.to_csv('AI_221')

## 🔥 도전과제

### 4) 다른 종류의 imputer를 사용하고 특성-타겟 관계 그래프를 나타내 보세요.
- 다른 종류의 imputer를 사용해 보세요([scikit-learn imputers](https://scikit-learn.org/stable/modules/impute.html))
- seaborn plots 사용하여 관심있는 특성들과 target간의 관계를 그래프로 나타내 보세요.
- [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html)
- [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html)

In [ ]:
### 이곳에서 과제를 진행해 주세요 ###